In [1]:
#!pip install carla

In [2]:
#!pip install -U ultralytics

In [3]:
import carla
import math
import random
import time
import numpy as np
import cv2
import pandas as pd

In [4]:
#The local Host for carla simulator is 2000
client = carla.Client('localhost', 2000)
# world has methods to access all things in simulator(vehicles, buildings, etc and spawning)
client.set_timeout(10.0)
world = client.get_world() 

In [5]:
#blueprint will acess to all bps to create objects( like vehicles, people)
bp_lib = world.get_blueprint_library()
#spawn points to spawnx`
spawn_points = world.get_map().get_spawn_points()

In [6]:
#List of vehicles in the blueprint
all_veh = []
for i in bp_lib:
    if "vehicle" in i.tags:
        id_parts = i.id.split(".")
        veh = id_parts[1]+'.'+id_parts[-1]
        all_veh.append(veh)

#all_veh

In [7]:
#From vehicle blueprint getting the information of specific vehicle
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
#vehicle_bp = bp_lib.filter('model3')[0]
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [8]:
#Shifting the view to the spectator of the car(camera)
spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x = -4, z = 2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [9]:
#Spawning the Vehicles as npc
spawn_num = 40
for i in range(spawn_num):
    spawn_vehicle = random.choice(bp_lib.filter('vehicle'))
    npc = world.try_spawn_actor(spawn_vehicle, random.choice(spawn_points))

In [10]:
#autopiloting the vehicles
for v in world.get_actors().filter('*vehicle*'):
    v.set_autopilot(True)

In [11]:
#Attaching camera sensor to the vehicle
camera_bp = bp_lib.find('sensor.camera.rgb')

#Setting the width and height to display
IM_WIDTH =  256*4
IM_HEIGHT =  256*3

camera_bp.set_attribute('image_size_x', f'{IM_WIDTH}')
camera_bp.set_attribute('image_size_y', f'{IM_HEIGHT}')
camera_bp.set_attribute('fov', '110')

#transforming the carla into a desirable position and attaching to the vehicle
camera_init_trans = carla.Transform(carla.Location(z=1.6, x = 0.4))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to = vehicle)

In [12]:
#To save images in the drive
#camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))

In [13]:
#To stop saving the images
#camera.stop()

In [14]:
#converting the image to use yolo
def camera_callback(image, data_dict):
    #data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    image_data = np.array(image.raw_data)
    image_rgb = image_data.reshape((image.height, image.width, 4))[:, :, :3]  # Extract RGB channels
    data_dict['image'] = image_rgb

In [15]:
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h, image_w, 4))}

camera.listen(lambda image:camera_callback(image, camera_data))

In [16]:
vehicle.set_autopilot(True)

In [17]:
img = camera_data['image']
print(img.shape)

(768, 1024, 3)


In [18]:
#destroying all the actors
def destroyAll():
    vehicle.destroy()
    camera.stop()
    camera.destroy()

    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()

    cv2.destroyAllWindows()

In [19]:
'''
from ultralytics import YOLO

#use the best model for object detection
model = YOLO('yolov8n.pt')
#model = YOLO("carla_ds.pt")

while True:
    frame = camera_data['image']
    results = model(frame ,show = True) 
    
    details = []
    for result in results:
        boxes = result.boxes.data
        #classes = result.cls
        #confidences = result.probs.squeeze()
        
        num_objects = len(boxes)

        for i in range(num_objects):
            box = boxes[i]
            #class_id = int(classes[i])
            #confidence = confidences[i]
            #details.append([box.tolist(), class_id, confidence.item()])
            details.append([box.tolist()])

    # Create the DataFrame
    columns = ['Box']
    yolo_details = pd.DataFrame(details, columns=columns)


    if cv2.waitKey(1) == ord('q'):
        destroyAll()
        break   


print(details)
'''

'\nfrom ultralytics import YOLO\n\n#use the best model for object detection\nmodel = YOLO(\'yolov8n.pt\')\n#model = YOLO("carla_ds.pt")\n\nwhile True:\n    frame = camera_data[\'image\']\n    results = model(frame ,show = True) \n    \n    details = []\n    for result in results:\n        boxes = result.boxes.data\n        #classes = result.cls\n        #confidences = result.probs.squeeze()\n        \n        num_objects = len(boxes)\n\n        for i in range(num_objects):\n            box = boxes[i]\n            #class_id = int(classes[i])\n            #confidence = confidences[i]\n            #details.append([box.tolist(), class_id, confidence.item()])\n            details.append([box.tolist()])\n\n    # Create the DataFrame\n    columns = [\'Box\']\n    yolo_details = pd.DataFrame(details, columns=columns)\n\n\n    if cv2.waitKey(1) == ord(\'q\'):\n        destroyAll()\n        break   \n\n\nprint(details)\n'

In [36]:

from ultralytics import YOLO
import time

model = YOLO('yolov8n.pt')


start_time = time.time()
threshold =0.3
details = []
yolo_df = []

model = YOLO('yolov8x.pt')

while True:
    frame = camera_data['image']
    results = model(frame ,show = True) #
   
    col = ['xyhw','x','y','w','h','conf','id']
    for box in results:  
        for r in box.boxes.data.tolist():
            x, y, w, h, conf, id = r
            print(r)
            xywh = [int((x + w) / 2), int((y + h) / 2), int(w - x), int(h - y)]
            details.append([xywh ,int(x), int(y), int(w), int(h), conf,id])
            print("details")
            print(details)


            
            yolo_df = pd.DataFrame(details, columns=col)
    print(yolo_df)

    if cv2.waitKey(1) == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()




0: 480x640 5 potted plants, 11130.6ms
Speed: 15.0ms preprocess, 11130.6ms inference, 13.0ms postprocess per image at shape (1, 3, 640, 640)



[568.6061401367188, 422.78179931640625, 636.0643920898438, 479.61090087890625, 0.8231167197227478, 58.0]
details
[[[602, 451, 67, 56], 568, 422, 636, 479, 0.8231167197227478, 58.0]]
[238.63916015625, 424.22735595703125, 330.9548034667969, 491.9462890625, 0.7674643993377686, 58.0]
details
[[[602, 451, 67, 56], 568, 422, 636, 479, 0.8231167197227478, 58.0], [[284, 458, 92, 67], 238, 424, 330, 491, 0.7674643993377686, 58.0]]
[803.6251220703125, 422.325439453125, 883.41162109375, 476.3934020996094, 0.7630200386047363, 58.0]
details
[[[602, 451, 67, 56], 568, 422, 636, 479, 0.8231167197227478, 58.0], [[284, 458, 92, 67], 238, 424, 330, 491, 0.7674643993377686, 58.0], [[843, 449, 79, 54], 803, 422, 883, 476, 0.7630200386047363, 58.0]]
[368.9820251464844, 1.990625023841858, 470.9801940917969, 574.7413330078125, 0.5891220569610596, 58.0]
details
[[[602, 451, 67, 56], 568, 422, 636, 479, 0.8231167197227478, 58.0], [[284, 458, 92, 67], 238, 424, 330, 491, 0.7674643993377686, 58.0], [[843, 449, 7

0: 480x640 5 potted plants, 12128.8ms
Speed: 14.0ms preprocess, 12128.8ms inference, 13.0ms postprocess per image at shape (1, 3, 640, 640)



[568.6486206054688, 423.51544189453125, 636.2645263671875, 479.7388610839844, 0.824436604976654, 58.0]
details
[[[602, 451, 67, 56], 568, 422, 636, 479, 0.8231167197227478, 58.0], [[284, 458, 92, 67], 238, 424, 330, 491, 0.7674643993377686, 58.0], [[843, 449, 79, 54], 803, 422, 883, 476, 0.7630200386047363, 58.0], [[419, 288, 101, 572], 368, 1, 470, 574, 0.5891220569610596, 58.0], [[728, 401, 39, 89], 708, 356, 748, 445, 0.551045298576355, 58.0], [[602, 451, 67, 56], 568, 423, 636, 479, 0.824436604976654, 58.0]]
[238.44859313964844, 425.0018615722656, 331.0226135253906, 492.1000061035156, 0.7911118268966675, 58.0]
details
[[[602, 451, 67, 56], 568, 422, 636, 479, 0.8231167197227478, 58.0], [[284, 458, 92, 67], 238, 424, 330, 491, 0.7674643993377686, 58.0], [[843, 449, 79, 54], 803, 422, 883, 476, 0.7630200386047363, 58.0], [[419, 288, 101, 572], 368, 1, 470, 574, 0.5891220569610596, 58.0], [[728, 401, 39, 89], 708, 356, 748, 445, 0.551045298576355, 58.0], [[602, 451, 67, 56], 568, 423,

0: 480x640 5 potted plants, 11355.9ms
Speed: 13.0ms preprocess, 11355.9ms inference, 11.7ms postprocess per image at shape (1, 3, 640, 640)



...

0: 480x640 5 potted plants, 11942.4ms
Speed: 15.0ms preprocess, 11942.4ms inference, 14.0ms postprocess per image at shape (1, 3, 640, 640)



[568.7174682617188, 423.4869079589844, 636.3815307617188, 479.66583251953125, 0.826421856880188, 58.0]
details
[[[602, 451, 67, 56], 568, 422, 636, 479, 0.8231167197227478, 58.0], [[284, 458, 92, 67], 238, 424, 330, 491, 0.7674643993377686, 58.0], [[843, 449, 79, 54], 803, 422, 883, 476, 0.7630200386047363, 58.0], [[419, 288, 101, 572], 368, 1, 470, 574, 0.5891220569610596, 58.0], [[728, 401, 39, 89], 708, 356, 748, 445, 0.551045298576355, 58.0], [[602, 451, 67, 56], 568, 423, 636, 479, 0.824436604976654, 58.0], [[284, 458, 92, 67], 238, 425, 331, 492, 0.7911118268966675, 58.0], [[843, 449, 80, 53], 803, 422, 883, 475, 0.7616778612136841, 58.0], [[727, 400, 39, 89], 708, 355, 747, 445, 0.5600794553756714, 58.0], [[420, 288, 102, 572], 368, 2, 471, 574, 0.5575940608978271, 58.0], [[602, 451, 67, 56], 568, 423, 636, 479, 0.8204577565193176, 58.0], [[284, 458, 92, 67], 238, 424, 330, 491, 0.7863591909408569, 58.0], [[843, 449, 80, 52], 803, 423, 883, 475, 0.7528225183486938, 58.0], [[419,

0: 480x640 5 potted plants, 10909.1ms
Speed: 14.0ms preprocess, 10909.1ms inference, 20.0ms postprocess per image at shape (1, 3, 640, 640)


[568.5445556640625, 423.2041015625, 636.0056762695312, 479.76953125, 0.8333120346069336, 58.0]
details
[[[602, 451, 67, 56], 568, 422, 636, 479, 0.8231167197227478, 58.0], [[284, 458, 92, 67], 238, 424, 330, 491, 0.7674643993377686, 58.0], [[843, 449, 79, 54], 803, 422, 883, 476, 0.7630200386047363, 58.0], [[419, 288, 101, 572], 368, 1, 470, 574, 0.5891220569610596, 58.0], [[728, 401, 39, 89], 708, 356, 748, 445, 0.551045298576355, 58.0], [[602, 451, 67, 56], 568, 423, 636, 479, 0.824436604976654, 58.0], [[284, 458, 92, 67], 238, 425, 331, 492, 0.7911118268966675, 58.0], [[843, 449, 80, 53], 803, 422, 883, 475, 0.7616778612136841, 58.0], [[727, 400, 39, 89], 708, 355, 747, 445, 0.5600794553756714, 58.0], [[420, 288, 102, 572], 368, 2, 471, 574, 0.5575940608978271, 58.0], [[602, 451, 67, 56], 568, 423, 636, 479, 0.8204577565193176, 58.0], [[284, 458, 92, 67], 238, 424, 330, 491, 0.7863591909408569, 58.0], [[843, 449, 80, 52], 803, 423, 883, 475, 0.7528225183486938, 58.0], [[419, 288, 10

KeyboardInterrupt: 